# Atomistic Simulation Tutorial

このチュートリアルは、原子シミュレーションを用いた材料開発を進めていくために必要な知識を、
コードレベルで実践的に学ぶことを目指して作成したオンラインの学習教材です。

## 対象読者

対象読者は以下のような方です。

 - 原子シミュレーションを専門とする方で、自分の専門とは別の計算に関する知識を勉強していきたい方
 - 原子シミュレーションではない理系の専門知識をお持ちで、これから原子シミュレーションも理解を深めて使いこなしていこうとする方
 - 実験家の方で、今後、実験結果の説明など理論面からもアプローチするために計算も行っていきたい方

このチュートリアルを一通り終えることで、以下に公開されているような各種計算事例や論文事例理解が出来るというレベルに進む手助けができれば理想です。

 - Matlantis計算事例・論文事例: https://matlantis.com/ja/cases/
 - MatlantisのContribリポジトリ:https://github.com/matlantis-pfcc/matlantis-contrib

## 本チュートリアルが扱う範囲


### 材料分野

材料開発というと、有機分子から金属・半導体など様々なものが存在します。<br/>
現時点でのスコープとしては、工業的に使用される固体材料によりフォーカスを置いた内容となっています。

材料分野としては例えば、以下のようなものが挙げられます。

 - 触媒
 - 電池材料
 - 金属
 - 半導体
 - セラミック
 - 有機分子
   - 潤滑油 
   - ポリマーなど
 
生体に作用するような分子は扱う原子数が大きく、異なったアプローチも必要となるため、現時点ではスコープ外となっています。


### シミュレーションのスケール

計算を用いた材料開発といっても、多くの手法が存在しそれぞれが扱う空間・時間スケールは様々です。

<figure style="max-width:640px">
<img src="../assets/time-length-scale-small.png"/>
</figure>

<cite>Created by modifying Fig. 1: Length and time-scale based computational materials design techniques from 
"[The joint automated repository for various integrated simulations (JARVIS) for data-driven materials design](https://www.nature.com/articles/s41524-020-00440-1)" ([Licensed under CC BY 4.0](https://creativecommons.org/licenses/by/4.0/))</cite>

本チュートリアルは上図の中でも、原子スケールをメインにフォーカスし従来の量子化学計算手法(DFTなど)や分子動力学法(Molecular Dynamics)で解析がされているような問題対象を扱います。
ただし、原子レベルでの現象を扱う "Atomistic simulation" にフォーカスし、電子状態の解析（半導体のバンドギャップ解析など）のようなミクロな解析は含みません。

## 機械学習を用いたシミュレーション技術
本チュートリアルは、商用ソフトウェアである[Matlantis](https://matlantis.com/ja/)のチュートリアルが元になっています。
Matlantisは、材料の原子同士の相互作用を近似するニューラルネットワークであるPreFerred Potential (PFP)をコアとし、超高速に物理シミュレーションを行えるソフトウェアです。[PFP](https://tech.preferred.jp/ja/blog/development-of-universal-neural-network-for-materials-discovery/) （[論文](https://www.nature.com/articles/s41467-022-30687-9)）のように、原子同士の相互作用をニューラルネットワークで近似する手法はNeural Network Potential（NNP）と呼ばれ、近年非常に活発に研究されています。

これまでNNPは特定の元素の組み合わせにしか対応できないなど、通常の第一原理計算と比べると適用対象に制約が大きいものでしたが、学習データの拡充とディープラーニング技術の進展を背景に、PFPやM3GNetをはじめとして、ほぼ制約なく様々な系に対して適用できる汎用的なNNPが登場しつつあります。このチュートリアルでは、C.Chenらによって開発された汎用NNPである[M3GNet](https://github.com/materialsvirtuallab/m3gnet)を用います。M3GNetはオープンソースソフトウェアであるため、誰でも無料で使えるほか、必要に応じて自分で拡張したり追加データで学習することもできます。

M3GNetは、材料オープンデータである[Materials Project](https://materialsproject.org/)に含まれる約19万件の結晶構造を用いて学習されており、予測モデルにはグラフニューラルネットワークが用いられています。ちなみに、原子をノード、結合をエッジとしたグラフで結晶構造を表現し、グラフニューラルネットワークを適用するというアプローチは、2018年に[CGCNN](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.120.145301)で提案されて以来、結晶構造をニューラルネットワークで扱う際のデファクトスタンダードとなっています。

## 原子シミュレーションを学ぶ意義

原子シミュレーションを用いることで、実験的に知られている現象の機構解明を行ったり、これまで想定していなかったような元素の組み合わせ・構造に対する解析を行い新材料開発を行ったりすることができます。<br/>
近年[SDGｓ (Sustainable Development Goals)](https://www.mofa.go.jp/mofaj/gaiko/oda/sdgs/index.html)といった言葉が使用されるようになり、世の中をただ便利にするだけではなく、持続可能な形でよりよい世界にしていくことを目指す活動が盛んになっています。<br/>
原子レベルシミュレーションを通した材料開発によって、持続可能な世界の実現に向けてアプローチすることのできる事例として、いくつかの具体例を上げてみます。

### アンモニア合成触媒

20世紀初頭、世界人口の爆発的な増加により食糧危機に際して、必要だったものは穀物の生産性を上げることでした。<br/>
穀物の生産に必要なもの、水・太陽の光・質の良い大地のうち、土の中の栄養は植物に吸い上げられてしまうため、
毎年の安定した収穫のためにはよい肥料が必要となります。

肥料には窒素の成分が必要になりますが、これまでは肥溜めなどの人工肥料を用いており大量生産が難しいという問題がありました。<br/>
このような社会状況の中、鉄を触媒として用いることで、空気中の窒素(N2)からアンモニア(NH3)を合成するというハーバー・ボッシュ法が確立され、
合成肥料を作ることが可能となりました。
つまり、ハーバー・ボッシュ法により「空気からパンを作る」ことが可能となり食料の工業的な生産が可能になりました。

1906年に開発されたこの方法は、1世紀以上が経過した現在でもベースとなる触媒材料は変わらずに様々な工業プロセスに使われていますが、
大量のエネルギーを消費する方法です。
世界で消費されているエネルギーの2%がハーバー・ボッシュ法に使用されているという報告もあり、
もし反応効率を改善できる触媒が見つかれば、その効果はとても大きなものとなります。

参考文献

 - 「分子レベルで見た触媒の働き　反応はなぜ速く進むのか」 松本吉泰
 - [The future is green: the future is ammonia… | World Fertilizer](https://www.worldfertilizer.com/special-reports/28122018/the-future-is-green-the-future-is-ammonia/)
 - [Reaction Mechanism and Kinetics for Ammonia Synthesis on the
Fe(111) Surface](https://pubs.acs.org/doi/10.1021/jacs.7b13409)


### 電池材料

スマホ・パソコン・電気自動車(Electric Vehicle)・家庭用蓄電池など、世の中での蓄電の需要はどんどん増しています。<br/>
リチウムイオンバッテリーを充放電する際には、リチウム(Li)が電解質を出たり入ったりするため、電池の中でLiがどれだけ速く動けるかが重要となります。
蓄電池の市場の中でも、安全性が高い全固体リチウムイオンバッテリーはその発展が期待されていますが、
Liイオンを高速で伝導させることが難しく、それが本格的な普及に向けた壁となっており、高いイオン伝導性を持つ固体電解質を開発することが重要となります。


以下の計算事例では、原子レベルでのMD(分子動力学)シミュレーションを行う事によりLiイオンの拡散係数が求められており、固体電解質の材料開発に役立つものと期待されています。

参考文献

 - [硫化物固体電解質中のLi拡散](https://matlantis.com/ja/calculation/li-diffusion-in-li10gep2s12-sulfide-solid-electrolyte)
 - [First Principles Study of the Li10GeP2S12 Lithium Super Ionic Conductor Material](https://pubs.acs.org/doi/10.1021/cm203303y)
 - [LIB材料世界市場、2023年に5兆7781億円規模へ：富士経済が調査](https://eetimes.itmedia.co.jp/ee/articles/2003/30/news049.html)
 - [全固体電池とは｜バッテリー(電池)基礎｜バッテリー技術](https://ev-tech.jp/technology/battery/page002_2.html)


### 再生可能エネルギー貯蔵のための水素・合成燃料生成触媒

脱炭素化に向けた取り組みとして再生可能エネルギーの利用が増えてきていますが、
太陽光発電は晴天・昼の間、風力発電も風が吹いている間と発電可能な時間が限られるため、
そのエネルギーを貯蔵していくことが、将来さらなる利用拡大のために必要となります。

このレベルの大規模なエネルギーを貯蔵するためには、充電効率の良い蓄電池だけではスケールが足りず、
既存施設を使った大規模蓄積が可能な水素やメタンなどに変換しての貯蔵が有望であるとされています。

現在、効率の高い変換を実現する触媒の発見が課題となっており、

 - 水(H2O)から水素(H2)や酸素(O2)を作る
 - 水素(H2)や二酸化炭素(CO2)から、メタン(CH4)などの合成燃料を作る

といった反応の効率が上げられるような触媒が見つかれば、
再生可能エネルギーを用いて得られたエネルギーを貯蔵し、
別の時間・場所で使うといったことができるようになることが期待されています。

以下の参考文献で、より詳しく説明がなされています。

参考文献

 - [[2010.09435] An Introduction to Electrocatalyst Design using Machine Learning for Renewable Energy Storage](https://arxiv.org/abs/2010.09435)
 - https://twitter.com/hillbig/status/1353481252389568512
 - [コバルト触媒におけるCO解離反応の活性化エネルギー](https://matlantis.com/ja/calculation/calculation-of-the-activation-barrier-for-co-dissociation-on-cobalt-fischer-tropsch-catalysts)
 - [The Vital Role of Step-Edge Sites for Both CO Activation and Chain Growth on Cobalt Fischer–Tropsch Catalysts Revealed through First-Principles-Based Microkinetic Modeling Including Lateral Interactions](https://pubs.acs.org/doi/full/10.1021/acscatal.0c02420)
 - [Coverage Effects in CO Dissociation on Metallic Cobalt Nanoparticles](https://pubs.acs.org/doi/full/10.1021/acscatal.9b01967)

<br/>


触媒上の反応解析については5章で、原子の拡散を扱うための手法である分子動力学法 (MD, Molecular Dynamics)については6章で紹介します。

もちろんここに上げた3例にとどまらず、[半導体](https://matlantis.com/ja/calculation/silicon-tma-tel)・[潤滑油](https://matlantis.com/ja/calculation/prediction-of-fluid-viscosity-of-alkanes-by-reverse-non-equilibrium-molecular-dynamics-simulation)・[吸着材](https://matlantis.com/ja/calculation/adsorption-and-dynamics-of-h2o-molecules-in-mof-74ni)、またそれ以外にもたくさんの応用先があります。

様々な材料に対してのシミュレーションを実施して頂き、そこから新たな知見が得られたり、革新的な材料の発見に貢献できれば幸いです。

## Pythonおよび主要ライブラリの使い方の学習

本チュートリアルではPythonプログラミングを使用し、Jupyter Lab上でコードの実行を進めていきます。<br/>

これまでプログラミングをしたことがない方や、Pythonでよく使われる主要ライブラリNumPyの使用に不安がある方は、[Chainer Tutorialの1. ~ 12.](https://tutorials.chainer.org/ja/tutorial.html)を学習することで勉強ができます。<br/>

1-2章でPython入門、3-7章はPythonを用いた化学計算ではよく使用する数学知識の勉強ができます。<br/>
８章以降は主要ライブラリの使い方の説明となっています。特に本チュートリアルで活用するものは太字で示しています。

本チュートリアルを読んでいて不明な点が出てきた場合など、必要に応じて勉強すると良いでしょう。

 - <b>[8. NumPy](https://tutorials.chainer.org/ja/08_Introduction_to_NumPy.html)：ベクトル計算を効率よく行うためのライブラリ。科学計算をおこなうPythonプログラミングでは頻出します。</b>
 - [9. scikit-learn](https://tutorials.chainer.org/ja/09_Introduction_to_Scikit-learn.html)：簡単に機械学習を行うためのライブラリ。本チュートリアルではあまり使用しませんが、Matlantisの計算結果・データを用いて機械学習を行う応用的な使い方をするときに役に立つでしょう。
 - <b>[11. Pandas](https://tutorials.chainer.org/ja/11_Introduction_to_Pandas.html)： 表形式にデータをまとめて、前処理・後処理・可視化などを行うことが出来るライブラリ。</b>
 - <b>[12. Matplotlib](https://tutorials.chainer.org/ja/12_Introduction_to_Matplotlib.html)： グラフ描画のためのライブラリ。</b>

ライブラリのドキュメントや参考論文などは、英語が書かれていることが多いです。英語に不慣れな方は[DeepL](https://www.deepl.com/ja/translator)や[Google 翻訳](https://translate.google.co.jp/?hl=ja)といった翻訳ツールも普及してきているので活用すると良いかもしれません。

それでは早速次章から実際にコードを動かしながら原子シミュレーションを学んでいきましょう。